In [1]:
import cv2
import numpy as np
from retinaface import RetinaFace
import face_recognition
import os
import csv
from datetime import datetime, timedelta
import threading

# Directory containing known faces
KNOWN_FACES_DIR = r"C:\attendance2\images"
TOLERANCE = 0.6  # Recognition tolerance
FRAME_THICKNESS = 3  # Thickness of bounding box
FONT_THICKNESS = 2  # Thickness of text
MODEL = "cnn"  # The model can be "hog" or "cnn" for detection

# Load known faces and their encodings
def load_known_faces():
    known_faces = []
    known_names = []

    for name in os.listdir(KNOWN_FACES_DIR):
        for filename in os.listdir(f"{KNOWN_FACES_DIR}/{name}"):
            image = face_recognition.load_image_file(f"{KNOWN_FACES_DIR}/{name}/{filename}")
            encodings = face_recognition.face_encodings(image)
            if encodings:
                known_faces.append(encodings[0])
                known_names.append(name)
    print("Loaded known faces")
    return known_faces, known_names

# Draw bounding boxes and names
def draw_faces(frame, faces, names):
    for (face, name) in zip(faces, names):
        # Draw a rectangle around the face
        cv2.rectangle(frame, (face[0], face[1]), (face[2], face[3]), (0, 255, 0), FRAME_THICKNESS)

        # Draw a label with a name below the face
        y_position = face[1] - 15 if face[1] - 15 > 15 else face[1] + 15
        cv2.rectangle(frame, (face[0], face[1]), (face[2], face[1] - 35), (0, 255, 0), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (face[0] + 6, y_position), font, 0.8, (255, 255, 255), FONT_THICKNESS)

# Recognize faces from live video
def recognize_faces_from_camera(camera_id, known_faces, known_names, tracking_dict):
    cap = cv2.VideoCapture(camera_id)
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Detect faces with RetinaFace
        faces = RetinaFace.detect_faces(frame)
        print(f"Frame recorded from Camera {camera_id}")

        # Prepare face encodings for recognition
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_locations = []
        face_names = []

        current_faces = set()  # To track currently detected faces

        for face_key in faces.keys():
            facial_area = faces[face_key]["facial_area"]

            # Append detected face locations for recognition
            face_locations.append(facial_area)
            
            current_faces.add(facial_area)
            
            # Convert RetinaFace coordinates to dlib format
            top, right, bottom, left = facial_area[1], facial_area[2], facial_area[3], facial_area[0]
            # Encode the face using face_recognition
            encodings = face_recognition.face_encodings(frame_rgb, [(top, right, bottom, left)])

            for encoding in encodings:
                # Compare with known faces
                matches = face_recognition.compare_faces(known_faces, encoding, TOLERANCE)
                name = "Unknown"

                # Find the shortest distance to a known face
                face_distances = face_recognition.face_distance(known_faces, encoding)
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = known_names[best_match_index]
                face_names.append(name)

                if name not in tracking_dict['seen']:
                    if camera_id == 0:
                        # Log entry time when a person is first detected by Camera 0
                        tracking_dict['entry_writer'].writerow([name, datetime.now().strftime('%Y-%m-%d %H:%M:%S')])
                        tracking_dict['seen'][name] = datetime.now()
                        tracking_dict['exit_seen'][name] = False
                    elif camera_id == 1:
                        # Log exit time when a person is first detected by Camera 1
                        if not tracking_dict['exit_seen'].get(name, False):
                            tracking_dict['exit_writer'].writerow([name, datetime.now().strftime('%Y-%m-%d %H:%M:%S')])
                            tracking_dict['exit_seen'][name] = True

        # Detect absent persons and log exit times if Camera 0
        if camera_id == 0:
            absent_names = set(tracking_dict['seen'].keys()) - set(face_names)
            for name in absent_names:
                if not tracking_dict['exit_seen'][name]:
                    if datetime.now() - tracking_dict['seen'][name] >= timedelta(minutes=2):
                        tracking_dict['exit_writer'].writerow([name, datetime.now().strftime('%Y-%m-%d %H:%M:%S')])
                        tracking_dict['exit_seen'][name] = True

        # Draw the results
        draw_faces(frame, face_locations, face_names)

        # Display the frame
        cv2.imshow(f'Camera {camera_id}', frame)

        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

def main():
    # Load known faces
    known_faces, known_names = load_known_faces()
    print("Loaded Done")
    print(known_names)

    # Open CSV files for logging
    with open(r'C:\attendance2\attendance_log.csv', mode='a', newline='') as file:
        entry_writer = csv.writer(file)
        exit_writer = csv.writer(file)

        tracking_dict = {
            'entry_writer': entry_writer,
            'exit_writer': exit_writer,
            'seen': {},  # To track when a person was last seen
            'exit_seen': {}  # To track if exit has been recorded for the person
        }

        # Start tracking with two cameras
        camera_ids = [0, 1]  # Adjust these based on your camera IDs
        threads = []

        for camera_id in camera_ids:
            thread = threading.Thread(target=recognize_faces_from_camera, args=(camera_id, known_faces, known_names, tracking_dict))
            thread.start()
            threads.append(thread)

        for thread in threads:
            thread.join()

if __name__ == "__main__":
    main()



Loaded known faces
Loaded Done
['akhil', 'sadhvika', 'shreyesh', 'shreyesh', 'srinath']
Frame recorded from Camera 0


Exception in thread Thread-6 (recognize_faces_from_camera):
Traceback (most recent call last):
  File "c:\Users\gshreyeshreddy\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\gshreyeshreddy\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\gshreyeshreddy\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\gshreyeshreddy\AppData\Local\Temp\ipykernel_18644\20586387.py", line 69, in recognize_faces_from_camera
TypeError: unhashable type: 'list'


Frame recorded from Camera 1
Frame recorded from Camera 0
Frame recorded from Camera 0
Frame recorded from Camera 0
Frame recorded from Camera 0
Frame recorded from Camera 0
